# Project 2
Keara Hayes, Atticus Chong, Zachary Malzahn, Evelyn Fuhrman

In [7]:
import numpy as np
from numpy.linalg import norm
import math
import matplotlib.pyplot as plt
import astropy.constants as _ac
import astropy.units as _au
from numpy import pi

## Old Project 1

# 1)

In [8]:
def rk4(f,t,z,h,args=()):
    """    
    Arguments
        f(t,z,...)
            function that contains the RHS of the equation dz/dt = f(t,z,...)
    
        t (scalar)
            current time
            
        z (array-like)
            function value
            
        h (scalar)
            step size
    
        args (tuple, optional)
            additional arguments to pass to f
    
    Returns
        znew = z(t+h)
    """
   
    if not isinstance(args,tuple):
        args = (args,)

    k1 = f(t, z, *args)

    k2 = f(t + h/2, z + (h/2)*k1, *args)

    k3 = f(t + h/2, z + (h/2)*k2, *args)

    k4 = f(t + h, z + h*k3, *args)
    
    return z + (h/6)*(k1 + 2*k2 + 2*k3 + k4)

# Project 2

part 1

In [9]:
###importing constants

# solar mass, radius, luminosity
Msun = _ac.M_sun.value
Rsun = _ac.R_sun.value
Lsun = _ac.L_sun.value

# physical constants from astropy, all in MKS units
G = _ac.G.value
h = _ac.h.value
hbar = _ac.hbar.value
m_e = _ac.m_e.value
m_p = _ac.m_p.value
m_n = _ac.m_n.value
m_u = _ac.u.value
c = _ac.c.value
kB = _ac.k_B.value
pc = _ac.pc.value
au = _ac.au.value
year = _au.year.to(_au.second)
sigmaSB = _ac.sigma_sb.value

# other constants
mue = 2
K_e = (1/5)*((3/(8*pi))**(2/3))*((h**2)/(2*m_e))

### part 2 :)

In [10]:
def pressure(rho, mue):
    """
    Arguments
        rho
            mass density (kg/m**3)
        mue
            baryon/electron ratio
    
    Returns
        electron degeneracy pressure (Pascal)
    """

#     p = (1/5) * ((3/(8*np.pi))**(2/3)) * ((h**2)/m_e) * ((rho/(mue * m_u))**(5/3)) # electron degeneracy pressure in Pa
    p = (1/5)*(3/(8*np.pi))**(2/3)*(h**2/m_e)*(rho/(mue*m_u))**(5/3)
    return p

In [11]:
def test_eos(pressure_func,density_func,mue=2.0,tolerance=1.0e-12,
        testfile='eos_table.txt',units='MKS'):
    """
    compares pressure and density functions against tabulated values

    Arguments
        pressure_func
            function that computes pressure given density and mue
        density_func
            function that computes density given pressure and mue
        mue
            baryon/electron ratio (default = 2.0)
        tolerance
            differences between computed and tabulated EOS values that are 
            larger than tolerance will trigger a warning (default = 1.0e-12)
        testfile
            table of density (kg/m**3), pressure (Pa), and Fermi energy 
            (electron rest mass-energy). Values computed by the eos module are              compared against this (default = 'eos_table.txt')
        units (either 'MKS' or 'CGS')
            specifies either MKS (default) or CGS units

    Returns
        True if all tests passed, False otherwise
    """

    accept = True
    
    # load data, ignoring column containing Fermi energy/(m_e c**2)
    data = np.loadtxt(testfile,skiprows=3,usecols=[0,1])
    density_val = data[:,0]
    pressure_val = data[:,1]

    # conversion factors if eos outputs cgs units
    density_mks_to_cgs = 1.0e-3
    pressure_mks_to_cgs = 10.0
    if units.lower()=='cgs':
        density_val *= density_mks_to_cgs
        pressure_val *= pressure_mks_to_cgs

    # table header
    print(
        '{0:>14}{1:>14}{2:>14}  {3:>14}{4:>14}{5:>14}\n'.format(
            'rho (table)','rho (test)','difference',
            'P (table)','P (test)','difference'))

    # iterate over density, pressure values and test individually
    for rho, p in zip(density_val, pressure_val):
        Ptest = pressure_func(rho,mue)
        rhotest = density_func(p,mue)
        Perr = (Ptest-p)/p
        rhoerr = (rhotest-rho)/rho
        print(
            '{0:14.6e}{1:14.6e}{2:14.6e}  {3:14.6e}{4:14.6e}{5:14.6e}'.\
            format(rho,rhotest,rhoerr,p,Ptest,Perr))

        if max(abs(Perr),abs(rhoerr)) > tolerance:
            print('******** value outside accepted tolerance ********\n')
            accept = False
            
    return accept


if __name__ == "__main__":

    print('Comparing EOS to eos_table.txt...\n')
    if (test_eos(pressure,density)):
        print('\nSUCCESS: all values within tolerance')
    else:
        print('\nFAIL')


Comparing EOS to eos_table.txt...

   rho (table)    rho (test)    difference       P (table)      P (test)    difference

  1.000000e+05  1.000000e+05  7.275958e-16    6.810445e+14  6.810445e+14  0.000000e+00
  1.123655e+05  1.123655e+05  2.590104e-16    8.271108e+14  8.271108e+14  7.556424e-16
  1.262600e+05  1.262600e+05  1.037282e-15    1.004504e+15  1.004504e+15 -7.466368e-16
  1.418727e+05  1.418727e+05  6.154215e-16    1.219944e+15  1.219944e+15  2.049274e-16
  1.594159e+05  1.594159e+05  3.651308e-16    1.481591e+15  1.481591e+15  6.749502e-16
  1.791284e+05  1.791284e+05  8.123732e-16    1.799353e+15  1.799353e+15 -1.389388e-16
  2.012785e+05  2.012785e+05  8.675688e-16    2.185268e+15  2.185268e+15 -3.432074e-16
  2.261676e+05  2.261676e+05  5.147303e-16    2.653950e+15  2.653950e+15  1.883984e-16
  2.541343e+05  2.541343e+05  5.726073e-16    3.223153e+15  3.223153e+15  0.000000e+00
  2.855592e+05  2.855592e+05  1.019187e-15    3.914436e+15  3.914436e+15 -3.831970e-16
  3.208

### part 3 :)

In [12]:
def stellar_derivatives(m,z,mue):
    """
    RHS of Lagrangian differential equations for radius and pressure
    
    Arguments
        m
            current value of the mass
        z (array)
            current values of (radius,pressure)
        mue
            ratio, nucleons to electrons. For a carbon-oxygen white dwarf, mue = 2.
        
    Returns
        dzdm (array)
            Lagrangian derivatives dr/dm, dP/dm
    """
    
    dzdm = np.zeros_like(z)
    
    dzdm[0] = (4*pi*(z[0]**2)*density(z[1],mue))**-1
    dzdm[1] = (-G*m)/(4*pi*(z[0]**4))
    
    return dzdm

part 4

In [13]:
def density(p, mue):
    """
    Arguments
        p
            electron degeneracy pressure (Pascal)
        mue
            baryon/electron ratio  
        mu
            amu
        h
            Planck's constant
        me
            electron mass
        
    Returns
        mass density (kg/m**3)
    """
    
#     rho = (p*(((1/5)*((3/8*pi)**2/3)*(h**2/(2*m_e)))**-1)**(3/5))*m_u*mue
    
    rho = mue*m_u*((5*p*(m_e/h**2)*(3/(8*np.pi))**(-2/3))**(3/5))
    return rho

In [14]:
density(1,2)

0.00012591999558046895

In [15]:
def central_values(Pc,delta_m,mue):
    """
    Constructs the boundary conditions at the edge of a small, constant density 
    core of mass delta_m with central pressure P_c
    
    Arguments
        Pc
            central pressure (units = ?)
        delta_m
            core mass (units = ?)
        mue
            nucleon/electron ratio
    
    Returns
        z = array([ r, p ])
            central values of radius and pressure (units = ?)
    """
    z = np.zeros(2)
    
    z[0] = ((3*delta_m)/(4 * np.pi * density(Pc, mue)))**1/3
    z[1] = Pc   
    return z

### part 5 :)

In [16]:
def lengthscales(m,z,mue):
    """
    Computes the radial length scale H_r and the pressure length H_P
    
    Arguments
        m
            current mass coordinate (units = ?)
        z (array)
           [ r, p ] (units = ?)
        mue
            mean electron weight
    
    Returns
        z/|dzdm| (units = ?)
    """    
    
    radius = z[0]
    pressure = z[1]
    rho = density(pressure, mue)
    H_r = 4 * np.pi * (radius**3) * rho
    H_p = (4 * np.pi * (radius**4) * pressure) / (G * m) 
    
    out = [H_r, H_p]
    
    out = np.asarray(out)
    
    return out

In [17]:
def integrate(Pc,delta_m,eta,xi,mue,max_steps=10000):
    """
    Integrates the scaled stellar structure equations

    Arguments
        Pc
            central pressure (units = ?)
        delta_m
            initial offset from center (units = ?)
        eta
            The integration stops when P < eta * Pc
        xi
            The stepsize is set to be xi*min(p/|dp/dm|, r/|dr/dm|)
        mue
            mean electron mass
        max_steps
            solver will quit and throw error if this more than max_steps are 
            required (default is 10000)
                        
    Returns
        m_step, r_step, p_step
            arrays containing mass coordinates, radii and pressures during 
            integration (what are the units?)
    """
        
    m_step = np.zeros(max_steps)
    r_step = np.zeros(max_steps)
    p_step = np.zeros(max_steps)
  
    # set starting conditions using central values
    z = central_values(Pc, delta_m, mue)
    temp = delta_m
    Nsteps = 0
    
    for step in range(max_steps):
        radius = z[0]
        pressure = z[1]
        
        # are we at the surface?
        if (pressure < eta*Pc):
            break
            
        # store the step
#         m_step[step] = m_step[Nsteps-1] + delta_m  ##just delm
        m_step[step] = temp
        r_step[step] = radius
        p_step[step] = pressure
        
        # set the stepsize
#         H_r, H_p = lengthscales(m_step[step], z, mue)
        h = xi*min(lengthscales(m_step[step], z, mue))

        # take a step
        z = rk4(stellar_derivatives,temp,z,h,args=(mue))
        temp += h
        
        # increment the counter
        Nsteps += 1
    # if the loop runs to max_steps, then signal an error
    else:
        raise Exception('too many iterations')
        
    return m_step[0:Nsteps],r_step[0:Nsteps],p_step[0:Nsteps]

# part 6 :)

In [18]:
def guess_P_c(M, mue):
    return ((G**5)/(K_e**4))*((M*(mue**2))**(10/3))

In [19]:
P_c_guess=guess_P_c(Msun, mue)

In [20]:
delta_m = 1e-4
eta = 1e-20
xi = 0.05

In [21]:
integrate(P_c_guess,delta_m,eta,xi,mue,max_steps=10000)

C:\Users\khaye\AppData\Local\Temp\ipykernel_12968\2337862741.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  dzdm[1] = (-G*m)/(4*pi*(z[0]**4))
C:\Users\khaye\AppData\Local\Temp\ipykernel_12968\1518563475.py:28: RuntimeWarning: invalid value encountered in multiply
  k2 = f(t + h/2, z + (h/2)*k1, *args)
C:\Users\khaye\AppData\Local\Temp\ipykernel_12968\1518563475.py:30: RuntimeWarning: invalid value encountered in multiply
  k3 = f(t + h/2, z + (h/2)*k2, *args)


Exception: too many iterations

part 7

# given arrays m, r, P

m_final = m[-1]
r_final = r[-1]
p_final = P[0]

def shooting(M_want, Pc_guess):
    
    diff = 1
    max_steps = 10000
    
    while (diff != 0):
        
        # do stuff, output M_new
        
        diff = M_want - M_new
        
        if diff < 0:
            
            Pc_new = 0.5*(Pc_guess + 0.5*Pc_guess)
            Pc_guess = Pc_new
            
        else:
            
            Pc_new = 0.5*(Pc_guess - 0.5*Pc_guess)
            Pc_guess = Pc_new
            
    return shit_idk

### part 8 :)

In [ ]:
import matplotlib.pyplot as plt
from observations import MassRadiusObservations

obs = MassRadiusObservations()

print(f"Masses:\n {obs.masses}\n\nRadii:\n {obs.radii}\n\nRadius Errors:\n {obs.radius_errors}\n\n")

for source, info in obs.sources.items():
    print('{0:20} M = {1:5.3f} +/- {2:5.3f} Msun'.format(source, info.mass, info.mass_error))

In [ ]:
# plt.plot(mass_stuff, radius_stuff, label = 'Our shit')
plt.scatter(obs.masses, obs.radii, label = 'Joyce et al. 2018')
plt.errorbar(obs.masses, obs.radii, xerr=obs.mass_errors, yerr=obs.radius_errors, fmt='o', ecolor = ['green', 'red'])

plt.title(f'~cheese~ :)\n', fontsize=15)
axes_fontsize = 12
plt.xlabel(r'$M/M_{\odot}$', fontsize = axes_fontsize);
plt.ylabel(r'$R/R_{\odot}$', fontsize = axes_fontsize);
plt.legend();